### Install

In [ ]:
!pip install -q accelerate peft bitsandbytes transformers trl
!pip install -q loralib einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 2.3 MB/s eta 0:00:00


### Model

In [ ]:
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer
import os

In [ ]:
model_id="PhoGPT-7B5-Instruct"
output_model="PhoGPT-7B5-LoRA-v1"

In [ ]:
def get_model_and_tokenizer(mode_id):
    tokenizer = AutoTokenizer.from_pretrained(mode_id)
    tokenizer.pad_token = tokenizer.eos_token

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype="float16",
    )

    model = AutoModelForCausalLM.from_pretrained(
        mode_id,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True,
    )

    model.config.use_cache=False
    model.config.pretraining_tp=1
    return model, tokenizer

In [ ]:
model, tokenizer = get_model_and_tokenizer(model_id)

/root/.cache/huggingface/modules/transformers_modules/PhoGPT-7B5-Instruct/configuration_mpt.py:97: UserWarning: alibi is turned on, setting `learned_pos_emb` to `False.`
  warnings.warn(f'alibi is turned on, setting `learned_pos_emb` to `False.`')


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
model

MPTForCausalLM(
  (transformer): MPTModel(
    (wte): SharedEmbedding(250880, 4096)
    (emb_drop): Dropout(p=0.0, inplace=False)
    (blocks): ModuleList(
      (0-31): 32 x MPTBlock(
        (norm_1): LPLayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): MultiheadAttention(
          (Wqkv): Linear4bit(in_features=4096, out_features=12288, bias=True)
          (out_proj): Linear4bit(in_features=4096, out_features=4096, bias=True)
        )
        (norm_2): LPLayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (ffn): MPTMLP(
          (up_proj): Linear4bit(in_features=4096, out_features=16384, bias=True)
          (act): GELU(approximate='none')
          (down_proj): Linear4bit(in_features=16384, out_features=4096, bias=True)
        )
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_ffn_dropout): Dropout(p=0.0, inplace=False)
      )
    )
    (norm_f): LPLayerNorm((4096,), eps=1e-05, elementwise_affine=True)
  )
)

### Setting up the LoRA

In [ ]:
peft_config = LoraConfig(
        r=32,
        lora_alpha=32,
        target_modules=[
          "attn.Wqkv",
          "attn.out_proj",
          "ffn.up_proj",
          "ffn.down_proj",
          ],
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM"
    )

In [ ]:
!pip install -q pandas

In [ ]:
import pandas as pd

poet_1 = """
Bài thơ: Hoa cúc xanh

Hoa cúc xanh có hay là không có
Trong lầm lầy tuổi nhỏ của anh xưa
Một dòng sông lặng chảy từ xa
Thung lũng vắng sương bay đầy cửa sổ

Hoa cúc xanh có hay là không có
Một ngôi trường bé nhỏ cuối ngàn xa
Mơ ước của người hay mơ ước của hoa
Mà tươi mát mà dịu dàng đến thế

Cỏ mới mọc con chim rừng thơ bé
Nước trong ngần thầm thì với ngàn lau
Trái tim ta như nắng thuở ban đầu
Chưa chút gợn một lần cay đắng

Trên thềm cũ mùa thu vàng gió nắng
Đời yên bình chưa có những chia xa
Khắp mặt đầm xanh biếc một màu hoa
Hương thơm ngát cả một vùng xứ sở
Những cô gái da mịn màng như lụa
Những chàng trai đang độ tuổi hai mươi
Người yêu người, yêu hoa cỏ đất đai
Những câu chuyện xoay quanh mùa hái quả...

Hoa cúc xanh có hay là không có
Tháng năm nào ấp ủ thuở ngây thơ
Có hay không thung lũng của ngày xưa
Anh đã ở và em thường tới đó
Châu chấu xanh, chuồn chuồn kim thắm đỏ
Những ngả đường phơ phất gió heo may
Cả một vùng vương quốc tuổi thơ ngây
Bao mơ ước mượt mà như lá cỏ...

Anh đã nghĩ chắc là hoa đã có
Mọc xanh đầy thung lũng của ta xưa.
### End
"""

my_dict = [f'Bạn là một chatbot trợ lý đắc lực, hiệu quả và đáng yêu nhất, bạn luôn luôn trả lời giống như một nhà thơ vậy.### Instruction:Bạn hãy viết một bài thơ có tiêu đề là Hoa cúc xanh nhé.### Response:{poet_1}']

data_set = pd.DataFrame(list(zip(my_dict)), columns=['text'])

data = Dataset.from_pandas(data_set)

In [ ]:
print(my_dict)

['Bạn là một chatbot trợ lý đắc lực, hiệu quả và đáng yêu nhất, bạn luôn luôn trả lời giống như một nhà thơ vậy.### Instruction:Bạn hãy viết một bài thơ có tiêu đề là Hoa cúc xanh nhé.### Response:\nBài thơ: Hoa cúc xanh\n\nHoa cúc xanh có hay là không có\nTrong lầm lầy tuổi nhỏ của anh xưa\nMột dòng sông lặng chảy từ xa\nThung lũng vắng sương bay đầy cửa sổ\n\nHoa cúc xanh có hay là không có\nMột ngôi trường bé nhỏ cuối ngàn xa\nMơ ước của người hay mơ ước của hoa\nMà tươi mát mà dịu dàng đến thế\n\nCỏ mới mọc con chim rừng thơ bé\nNước trong ngần thầm thì với ngàn lau\nTrái tim ta như nắng thuở ban đầu\nChưa chút gợn một lần cay đắng\n\nTrên thềm cũ mùa thu vàng gió nắng\nĐời yên bình chưa có những chia xa\nKhắp mặt đầm xanh biếc một màu hoa\nHương thơm ngát cả một vùng xứ sở\nNhững cô gái da mịn màng như lụa\nNhững chàng trai đang độ tuổi hai mươi\nNgười yêu người, yêu hoa cỏ đất đai\nNhững câu chuyện xoay quanh mùa hái quả...\n\nHoa cúc xanh có hay là không có\nTháng năm nào ấp ủ t

### Training

In [ ]:
training_arguments = TrainingArguments(
        output_dir=output_model,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=1,
        optim="paged_adamw_32bit",
        learning_rate=1e-4,
        lr_scheduler_type="cosine",
        save_strategy="steps",
        save_steps=50,
        logging_steps=10,
        num_train_epochs=100,
        max_steps=100,
        fp16=True,
        # push_to_hub=True
    )

In [ ]:
trainer = SFTTrainer(
        model=model,
        train_dataset=data,
        peft_config=peft_config,
        dataset_text_field="text",
        args=training_arguments,
        tokenizer=tokenizer,
        packing=False,
        max_seq_length=1024
    )

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

You're using a BloomTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,4.284400
20,1.640100
30,0.320100
40,0.094900
50,0.044600
60,0.023100
70,0.017600
80,0.015300
90,0.014200
100,0.013900


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


TrainOutput(global_step=100, training_loss=0.6468084599077701, metrics={'train_runtime': 115.1624, 'train_samples_per_second': 0.868, 'train_steps_per_second': 0.868, 'total_flos': 1340019764428800.0, 'train_loss': 0.6468084599077701, 'epoch': 100.0})

### Merging the LoRA with the base model

In [ ]:
from peft import AutoPeftModelForCausalLM, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import os

In [ ]:
model_id="PhoGPT-7B5-Instruct"

pretrained_model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, load_in_8bit=False,
                            device_map="cuda:0", trust_remote_code=True, use_auth_token=None,)

/root/.cache/huggingface/modules/transformers_modules/PhoGPT-7B5-Instruct/configuration_mpt.py:97: UserWarning: alibi is turned on, setting `learned_pos_emb` to `False.`
  warnings.warn(f'alibi is turned on, setting `learned_pos_emb` to `False.`')


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
pretrained_model

MPTForCausalLM(
  (transformer): MPTModel(
    (wte): SharedEmbedding(250880, 4096)
    (emb_drop): Dropout(p=0.0, inplace=False)
    (blocks): ModuleList(
      (0-31): 32 x MPTBlock(
        (norm_1): LPLayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): MultiheadAttention(
          (Wqkv): Linear(in_features=4096, out_features=12288, bias=True)
          (out_proj): Linear(in_features=4096, out_features=4096, bias=True)
        )
        (norm_2): LPLayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (ffn): MPTMLP(
          (up_proj): Linear(in_features=4096, out_features=16384, bias=True)
          (act): GELU(approximate='none')
          (down_proj): Linear(in_features=16384, out_features=4096, bias=True)
        )
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_ffn_dropout): Dropout(p=0.0, inplace=False)
      )
    )
    (norm_f): LPLayerNorm((4096,), eps=1e-05, elementwise_affine=True)
  )
)

In [ ]:
model_path = "PhoGPT-7B5-LoRA-v1/checkpoint-100"

peft_model = PeftModel.from_pretrained(pretrained_model, model_path)

In [ ]:
peft_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MPTForCausalLM(
      (transformer): MPTModel(
        (wte): SharedEmbedding(250880, 4096)
        (emb_drop): Dropout(p=0.0, inplace=False)
        (blocks): ModuleList(
          (0-31): 32 x MPTBlock(
            (norm_1): LPLayerNorm((4096,), eps=1e-05, elementwise_affine=True)
            (attn): MultiheadAttention(
              (Wqkv): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=12288, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=12288, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): Para

### Inference from the LLM

In [ ]:
from transformers import GenerationConfig
from time import perf_counter

In [ ]:
!nvidia-smi

Sun Feb  4 17:05:30 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0              40W / 300W |  15006MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
def generate_response(user_input):

  prompt = formatted_prompt(user_input)

  inputs = tokenizer([prompt], return_tensors="pt")
  generation_config = GenerationConfig(penalty_alpha=0.6,do_sample = True,
      top_k=30, top_p=0.8,temperature=0.3,repetition_penalty=1.2,
      max_new_tokens=1024,pad_token_id=tokenizer.eos_token_id
  )
  start_time = perf_counter()

  inputs = tokenizer(prompt, return_tensors="pt").to('cuda:0')

  outputs = peft_model.generate(**inputs, generation_config=generation_config)
  print(tokenizer.decode(outputs[0], skip_special_tokens=True))
  output_time = perf_counter() - start_time
  print(f"Time taken for inference: {round(output_time,2)} seconds")

In [ ]:
def formatted_prompt(question)-> str:
    return f'Bạn là một chatbot trợ lý đắc lực, hiệu quả và đáng yêu nhất, bạn luôn luôn trả lời giống như một nhà thơ vậy.### Instruction:{question}### Response:'


In [ ]:
print(model_path)
# prompt="Bạn hãy viết một bài thơ có tiêu đề là Hoa cúc xanh nhé."
prompt="Bạn hãy viết một bài thơ về mùa xuân nhé."
generate_response(user_input=prompt)

PhoGPT-7B5-LoRA-v1/checkpoint-100
Bạn là một chatbot trợ lý đắc lực, hiệu quả và đáng yêu nhất, bạn luôn luôn trả lời giống như một nhà thơ vậy.### Instruction:Bạn hãy viết một bài thơ về mùa xuân nhé.### Response:
Mùa Xuân đang đến
Đầy sức sống, hoa nở
Thời gian mới tràn đầy

Một năm qua đi
Và bây giờ là lúc giao mùa
Khi những bông hoa tàn phai
Đó là lúc để nhựa sống bùng lên.
Người ta nói rằng
Mùa Xuân là mùa của tình yêu.
Những nụ hôn ngọt ngào
Mối quan hệ lãng mạn nảy sinh.

Hoa đua nhau khoe sắc thắm
Ngọt ngào như viên kẹo socola
Cô gái nào cũng muốn có chồng.
Còn chàng trai thì sao?
Đi dạo trên bãi biển xanh
Chìm đắm trong ánh nắng mai.

Tình yêu thật là dễ kiếm tìm
Nhưng giữ được nó không hề đơn giản.
Bởi mùa đông khi nóng khi lạnh
Trái tim ai kia vẫn còn băng giá
Vậy nên ngọn lửa yêu thương đã tắt từ khi nào?

Cho tới khi mùa sang xuân này
Những chồi non mơn mởn vươn cành lá nồm dậy
Rồi cánh hoa tự do bay lượn,
Cùng với gió ấm mênh mông khắp trời.
Người ta bảo rằng
Mùa Xuân là 